In [1]:
!pip install langchain faiss-cpu chromadb transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.2 MB/s eta 0:00:00


In [3]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
!git clone https://github.com/Gosha69228/TEXT.git


Cloning into 'TEXT'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 42 (delta 15), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 33.57 KiB | 498.00 KiB/s, done.
Resolving deltas: 100% (15/15), done.


In [5]:
import sqlite3

conn = sqlite3.connect('knowledge_base.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS articles (
    id INTEGER PRIMARY KEY,
    title TEXT,
    content TEXT
)
''')
conn.commit()

In [6]:
import os

repo_path = '/content/TEXT'

for filename in os.listdir(repo_path):
    if filename.endswith('.txt'):
        with open(os.path.join(repo_path, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            title = filename[:-4]  # Имя файла без расширения
            cursor.execute('INSERT INTO articles (title, content) VALUES (?, ?)', (title, content))

conn.commit()


In [7]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.3 MB/s eta 0:00:00


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
conn = sqlite3.connect('knowledge_base.db')
cursor = conn.cursor()

cursor.execute("SELECT id, content FROM articles")
articles = cursor.fetchall()

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

texts = [article[1] for article in articles]
ids = [str(article[0]) for article in articles]  # Преобразуем ID в строки


vectorstore = Chroma.from_texts(texts, embeddings, ids=ids)

conn.close()

<ipython-input-9-dae495a95168>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a tok

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
!pip install langchain transformers torch

In [12]:
!pip install --upgrade langchain


In [18]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
!pip install -U langchain-huggingface


In [19]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

# Инициализируем модель через pipeline
llm = HuggingFacePipeline(pipeline=pipeline("text-generation", model="distilgpt2", device=0))  # Указываем device=0 для использования GPU


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
from langchain.chains import create_retrieval_chain


In [26]:
# Инициализируем модель через pipeline
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model="distilgpt2",
        device=0,
        max_length=50,  # Максимальная длина генерируемой последовательности
        max_new_tokens=50  # Максимальное количество новых токенов для генерации
    )
)

# Создаем цепочку RAG
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Можно использовать "map_reduce", "stuff" и другие, в зависимости от желаемого подхода
    retriever=vectorstore.as_retriever()
)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [31]:
query = "Как защититься от спама?"
response = rag_chain.invoke(query)

print("Ответ:", response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=50) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Ответ: {'query': 'Как защититься от спама?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nЧто делать, если компьютер заражен?\nК сожалению, иногда случается так, что установленный в системе антивирус с самыми последними обновлениями не в состоянии обнаружить новый вирус, червя или троянскую программу. Увы — 100% безопасности не гарантирует ни одна антивирусная защита. В этом случае необходимо определить факт заражения, обнаружить вирусный файл и отправить его в антивирусную компанию, продукт которой «проморгал» вредную программу и не смог защитить компьютер от заражения.\n\nОднако в большинстве случаев самостоятельно (без помощи антивирусных программ) заметить факт заражения компьютера достаточно сложно — многие черви и троянские программы никак не проявляют своего присутствия. Бывают, конечно, случаи, когда троянцы явно сообщают пользователю, что компьюте